In [11]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding,GRU, Dropout, Flatten, concatenate, Convolution1D, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import codecs
from keras.optimizers import SGD, Adam, RMSprop
import io
import random as rn
import tensorflow as tf
from keras.layers.core import Dense, Activation
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
# x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape
# remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_mocap = x_train_mocap[mask]


(10039, 200, 189)

In [16]:
# API model, if use RNN, first two rnn layer must return_sequences=True
def api_model(a,b,c):
    
    input_mocap = Input(shape=(200,189))
#     model_mocap1 = BatchNormalization()(input_mocap)
    model_mocap2 = LSTM(256, return_sequences=True)(input_mocap)
    model_mocap3 = LSTM(256, return_sequences=True)(model_mocap2)
    model_mocap4 = LSTM(256, return_sequences=False)(model_mocap3)
    model_mocap5 = Dense(64)(model_mocap4)
    
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_mocap5) for name in target_names]

    model = Model(input_mocap, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': a, 'a': b, 'd': c},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(1, 1, 1)
#odel = api_model(0.1, 0.5, 0.4)
#model.summary()
# main function for for LOSO


In [17]:
def main():
    model = api_model(1,1,1)
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_mocap[:7869], 
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_mocap[7869:], vad[7869:].T.tolist())
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.1, 0.5, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(10):
        best = main()
        weight.append([time, best[-3:], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('mocap_LSTM1.csv', header=['time', 'best', 'ave'])



Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 232s 37ms/step - loss: 2.8594 - v_loss: 0.9733 - a_loss: 0.9602 - d_loss: 0.9256 - v_ccc: 0.0260 - a_ccc: 0.0405 - d_ccc: 0.0742 - val_loss: 2.7708 - val_v_loss: 0.9978 - val_a_loss: 0.9027 - val_d_loss: 0.8726 - val_v_ccc: -3.2670e-04 - val_a_ccc: 0.1005 - val_d_ccc: 0.1290
Epoch 2/50
6295/6295 [==============================] - 229s 36ms/step - loss: 2.6948 - v_loss: 0.9303 - a_loss: 0.9001 - d_loss: 0.8641 - v_ccc: 0.0693 - a_ccc: 0.1000 - d_ccc: 0.1359 - val_loss: 2.7415 - val_v_loss: 0.9809 - val_a_loss: 0.8922 - val_d_loss: 0.8691 - val_v_ccc: 0.0142 - val_a_ccc: 0.1118 - val_d_ccc: 0.1325
Epoch 3/50
6295/6295 [==============================] - 228s 36ms/step - loss: 2.6742 - v_loss: 0.9153 - a_loss: 0.8986 - d_loss: 0.8598 - v_ccc: 0.0843 - a_ccc: 0.1013 - d_ccc: 0.1403 - val_loss: 2.8103 - val_v_loss: 0.9935 - val_a_loss: 0.8955 - val_d_loss: 0.9271 - val_v_ccc: 0.0066 - val

2170/2170 [==============================] - 27s 13ms/step
0
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 263s 42ms/step - loss: 2.8431 - v_loss: 0.9658 - a_loss: 0.9489 - d_loss: 0.9289 - v_ccc: 0.0344 - a_ccc: 0.0511 - d_ccc: 0.0713 - val_loss: 2.8886 - val_v_loss: 1.0014 - val_a_loss: 0.9361 - val_d_loss: 0.9520 - val_v_ccc: -0.0033 - val_a_ccc: 0.0661 - val_d_ccc: 0.0486
Epoch 2/50
6295/6295 [==============================] - 263s 42ms/step - loss: 2.7058 - v_loss: 0.9271 - a_loss: 0.9046 - d_loss: 0.8743 - v_ccc: 0.0728 - a_ccc: 0.0956 - d_ccc: 0.1258 - val_loss: 2.7455 - val_v_loss: 0.9838 - val_a_loss: 0.8936 - val_d_loss: 0.8700 - val_v_ccc: 0.0118 - val_a_ccc: 0.1112 - val_d_ccc: 0.1316
Epoch 3/50
6295/6295 [==============================] - 264s 42ms/step - loss: 2.7001 - v_loss: 0.9268 - a_loss: 0.9045 - d_loss: 0.8690 - v_ccc: 0.0733 - a_ccc: 0.0953 - d_ccc: 0.1312 - val_loss: 2.7507 - val_v_loss: 0.9872 - val_a_los

6295/6295 [==============================] - 268s 43ms/step - loss: 2.7081 - v_loss: 0.9328 - a_loss: 0.9045 - d_loss: 0.8712 - v_ccc: 0.0673 - a_ccc: 0.0956 - d_ccc: 0.1290 - val_loss: 2.7444 - val_v_loss: 0.9869 - val_a_loss: 0.8931 - val_d_loss: 0.8662 - val_v_ccc: 0.0091 - val_a_ccc: 0.1111 - val_d_ccc: 0.1354
Epoch 3/50
6295/6295 [==============================] - 267s 42ms/step - loss: 2.6852 - v_loss: 0.9304 - a_loss: 0.8920 - d_loss: 0.8632 - v_ccc: 0.0697 - a_ccc: 0.1081 - d_ccc: 0.1371 - val_loss: 2.7519 - val_v_loss: 0.9894 - val_a_loss: 0.8912 - val_d_loss: 0.8733 - val_v_ccc: 0.0070 - val_a_ccc: 0.1129 - val_d_ccc: 0.1282
Epoch 4/50
6295/6295 [==============================] - 268s 43ms/step - loss: 2.6812 - v_loss: 0.9256 - a_loss: 0.8924 - d_loss: 0.8637 - v_ccc: 0.0745 - a_ccc: 0.1078 - d_ccc: 0.1365 - val_loss: 2.7539 - val_v_loss: 0.9880 - val_a_loss: 0.8919 - val_d_loss: 0.8756 - val_v_ccc: 0.0081 - val_a_ccc: 0.1120 - val_d_ccc: 0.1260
Epoch 5/50
6295/6295 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 271s 43ms/step - loss: 2.1739 - v_loss: 0.5273 - a_loss: 0.7982 - d_loss: 0.8482 - v_ccc: 0.4726 - a_ccc: 0.2018 - d_ccc: 0.1516 - val_loss: 2.6810 - val_v_loss: 0.9311 - val_a_loss: 0.8859 - val_d_loss: 0.8685 - val_v_ccc: 0.0685 - val_a_ccc: 0.1173 - val_d_ccc: 0.1331
Epoch 18/50
6295/6295 [==============================] - 269s 43ms/step - loss: 2.1810 - v_loss: 0.5327 - a_loss: 0.7977 - d_loss: 0.8511 - v_ccc: 0.4676 - a_ccc: 0.2022 - d_ccc: 0.1492 - val_loss: 2.6908 - val_v_loss: 0.9110 - val_a_loss: 0.8968 - val_d_loss: 0.8755 - val_v_ccc: 0.0776 - val_a_ccc: 0.1058 - val_d_ccc: 0.1258
Epoch 19/50
6295/6295 [==============================] - 269s 43ms/step - loss: 2.1752 - v_loss: 0.5135 - a_loss: 0.8003 - d_loss: 0.8608 - v_ccc: 0.4865 - a_ccc: 0.1993 - d_ccc: 0.1391 - val_loss: 2.6704 - val_v_loss: 0.9189 - val_a_loss: 0.8865 - val_d_loss: 0.8670 - val_v_ccc: 0.0783 - val_a_ccc: 0.1167 - val_d_ccc: 0.1347
Epoch 20/50
6295/6295 [=====

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 250s 40ms/step - loss: 2.4119 - v_loss: 0.6843 - a_loss: 0.8726 - d_loss: 0.8554 - v_ccc: 0.3160 - a_ccc: 0.1272 - d_ccc: 0.1449 - val_loss: 2.6689 - val_v_loss: 0.8832 - val_a_loss: 0.9040 - val_d_loss: 0.8862 - val_v_ccc: 0.1177 - val_a_ccc: 0.0981 - val_d_ccc: 0.1153
Epoch 6/50
6295/6295 [==============================] - 244s 39ms/step - loss: 2.3648 - v_loss: 0.6460 - a_loss: 0.8599 - d_loss: 0.8587 - v_ccc: 0.3544 - a_ccc: 0.1395 - d_ccc: 0.1412 - val_loss: 2.6959 - val_v_loss: 0.9328 - val_a_loss: 0.8920 - val_d_loss: 0.8702 - val_v_ccc: 0.0609 - val_a_ccc: 0.1119 - val_d_ccc: 0.1312
Epoch 7/50
6295/6295 [==============================] - 227s 36ms/step - loss: 2.3045 - v_loss: 0.6022 - a_loss: 0.8397 - d_loss: 0.8628 - v_ccc: 0.3981 - a_ccc: 0.1602 - d_ccc: 0.1371 - val_loss: 2.7210 - val_v_loss: 0.9462 - val_a_loss: 0.9007 - val_d_loss: 0.8774 - val_v_ccc: 0.0526 - val_a_ccc: 0.1022 - val_d_ccc: 0.1241
Epoch 8/50
4736/6295 [========

KeyboardInterrupt: 